In [1]:
!pip install pdfkit reportlab google-cloud-storage langchain langchain-community langchain-google-vertexai google-cloud-bigquery google-cloud-bigquery-storage pandas pyarrow google-auth db-dtypes google-cloud-aiplatform

In [2]:
from google.colab import auth
auth.authenticate_user()

import pandas as pd
import matplotlib.pyplot as plt
from google.cloud import bigquery
from google.cloud import aiplatform
from langchain.prompts import PromptTemplate
from langchain_google_vertexai import VertexAI
import html
from datetime import date, datetime
from IPython.display import HTML, display
import re
from io import BytesIO
import io
import decimal
from google.api_core import exceptions
from google.cloud import bigquery, aiplatform


In [3]:


PROJECT_ID = "wagon-bootcamp-406121"
LOCATION = "us-central1"
DATASET_ID = 'xero_xero'
MODEL_NAME = 'gemini-2.0-flash-001'

bq_client = bigquery.Client(project=PROJECT_ID)
aiplatform.init(project=PROJECT_ID, location=LOCATION)



In [4]:
def initialize_clients():
  """Initialize BigQuery and Vertex AI clients using global PROJECT_ID"""
  from google.cloud import bigquery, aiplatform
  from langchain.llms import VertexAI

  aiplatform.init(project=PROJECT_ID, location=LOCATION)

  bigquery_client = bigquery.Client(project=PROJECT_ID)

  llm = VertexAI(
    model_name=MODEL_NAME,
    max_output_tokens=2048,
    temperature=0,
    top_p=0.9,
    top_k=30,
    project=PROJECT_ID,
    location=LOCATION,
    verbose=True,
  )

  return bigquery_client, llm


In [6]:
# Utility Functions

def decimal_to_float(value):
  """Convert Decimal to float."""
  return float(value) if isinstance(value, Decimal) else value

def safe_percentage(part, whole):
  """Safely calculate percentage."""
  return (part / whole * 100) if whole != 0 else 0

def format_currency(value):
  """Format value as currency."""
  return f"£{value:,.2f}"

def calculate_percentage_change(current, previous):
  """Calculate percentage change."""
  if previous == 0:
    return float('inf') if current > 0 else float('-inf') if current < 0 else 0
  return (current - previous) / abs(previous) * 100

# Initialize clients
def initialize_clients():
  """Initialize BigQuery and Vertex AI clients."""
  bigquery_client = bigquery.Client(project=PROJECT_ID)
  aiplatform.init(project=PROJECT_ID, location=LOCATION)
  llm = VertexAI(
    model_name=MODEL_NAME,
    max_output_tokens=2048,
    temperature=0, #A good starting point. This range encourages a focused, informative tone while allowing for some natural language variation
    top_p=0.9, #This narrower range prioritizes highly probable and relevant words, making the summary less prone to hallucinations or off-topic tangents.
    top_k=30, #Top-k sampling limits the vocabulary the model considers, but a narrower range might not be necessary for this task.
    project=PROJECT_ID,
    location=LOCATION,
    verbose=True,
  )
  return bigquery_client, llm

In [7]:
def fetch_data(client: bigquery.Client, query: str):
    """Fetches data from BigQuery given a client and a query string."""
    query_job = client.query(query)
    results = query_job.result()
    return results.to_dataframe()

In [8]:


def get_pl_data(client):
  """Retrieve P&L data from BigQuery."""
  pl_query = f"""
  WITH
    report AS (
    SELECT
      date_month,
      CASE
        WHEN account_type = 'REVENUE' THEN 'Turnover'
        WHEN account_type = 'OVERHEADS' THEN 'Overheads'
        WHEN account_type = 'DIRECTCOSTS' THEN 'Direct Costs'
    END
      AS account_category,  -- Changed from account_report_category to account_category
      CASE
        WHEN account_type = 'REVENUE' THEN 1
        WHEN account_type = 'OVERHEADS' THEN 4
        WHEN account_type = 'DIRECTCOSTS' THEN 2
    END
      AS account_report_category_order,
      account_name AS account_group,
      account_id,
      account_code,
      account_class,
      account_type,
      COALESCE(SUM(net_amount* -1),0) AS net_amount
    FROM
      `{PROJECT_ID}.{DATASET_ID}.xero__profit_and_loss_report`
    GROUP BY
      1, 2, 3, 4, 5, 6, 7,8),
    spine AS (
    WITH
      rawdata AS (
      WITH
        p AS (
        SELECT
          0 AS generated_number
        UNION ALL
        SELECT
          1 ),
        unioned AS (
        SELECT
          p0.generated_number * POWER(2, 0) + p1.generated_number * POWER(2, 1) + p2.generated_number * POWER(2, 2) + p3.generated_number * POWER(2, 3) + p4.generated_number * POWER(2, 4) + p5.generated_number * POWER(2, 5) + p6.generated_number * POWER(2, 6) + 1 AS generated_number
        FROM
          p AS p0
        CROSS JOIN
          p AS p1
        CROSS JOIN
          p AS p2
        CROSS JOIN
          p AS p3
        CROSS JOIN
          p AS p4
        CROSS JOIN
          p AS p5
        CROSS JOIN
          p AS p6 )
      SELECT
        *
      FROM
        unioned
      WHERE
        generated_number <= 68
      ORDER BY
        generated_number ),
      all_periods AS (
      SELECT
        ( DATETIME_ADD( CAST( CAST('2023-06-01' AS date) AS datetime), INTERVAL ROW_NUMBER() OVER (ORDER BY 1) - 0 month ) ) AS date_month
      FROM
        rawdata ),
      filtered AS (
      SELECT
        *
      FROM
        all_periods
      WHERE
        date_month <= DATETIME_ADD( CAST( current_date AS datetime), INTERVAL 1 month ) )
    SELECT
      *
    FROM
      filtered ),
    cleaned AS (
    SELECT
      CAST(date_month AS date) AS date_month
    FROM
      spine ),
    calendar AS (
    SELECT
      *
    FROM
      cleaned),
    joined AS (
    SELECT
      calendar.date_month,
      report.account_id,
      report.account_group,
      report.account_code,
      report.account_class,
      report.account_type,
      report.account_category,  -- Changed from account_report_category to account_category
      report.account_report_category_order,
      COALESCE(SUM(net_amount * -1),0) AS net_amount
    FROM
      calendar
    LEFT JOIN
      report
    ON
      calendar.date_month = CAST( TIMESTAMP_TRUNC( CAST(report.date_month AS timestamp ), month ) AS date)
    GROUP BY
      1,2,3,4,5,6,7,8),
  report_with_subtotals_totals as (
  SELECT
    date_month,
    account_report_category_order,
    account_category,  -- Changed from account_report_category account_category to account_category
    SUM(net_amount) AS net_amount
  FROM
    joined
  WHERE
    account_category IS NOT NULL  -- Changed from account_report_category to account_category
  GROUP BY
    1, 2, 3
  UNION ALL
  SELECT
    *
  FROM (
    SELECT
      date_month,
      3 AS account_report_category_order,
      'Gross Profit' AS account_category,
      SUM(net_amount) OVER (PARTITION BY date_month) AS net_amount
    FROM
      joined
    WHERE
      account_category IN ('Turnover', 'Direct Costs'))  -- Changed from account_report_category to account_category
  GROUP BY
    1, 2, 3, 4
  UNION ALL
  SELECT
    *
  FROM (
    SELECT
      date_month,
      5 AS account_report_category_order,
      'Net Profit' AS account_category,
      SUM(net_amount) OVER (PARTITION BY date_month) AS net_amount
    FROM
      joined
    WHERE
      account_category IN ('Turnover', 'Direct Costs', 'Overheads'))  -- Changed from account_report_category to account_category
  UNION ALL
  SELECT
    *
  FROM (
    SELECT
      date_month,
      7 AS account_report_category_order,
      'Retained Earnings' AS account_category,
      SUM(net_amount) OVER (PARTITION BY date_month) * 0.75 AS net_amount
    FROM
      joined
    WHERE
      account_category IN ('Turnover', 'Direct Costs', 'Overheads'))  -- Changed from account_report_category to account_category
  UNION ALL
  SELECT
    *
  FROM (
    SELECT
      date_month,
      6 AS account_report_category_order,
      'Taxation' AS account_category,
      SUM(net_amount) OVER (PARTITION BY date_month) * 0.25 AS net_amount
    FROM
      joined
    WHERE
      account_category IN ('Turnover', 'Direct Costs', 'Overheads'))  -- Changed from account_report_category to account_category
  GROUP BY
    1, 2, 3, 4
  ORDER BY
    date_month,
    account_report_category_order)
  SELECT
    *
  FROM
   report_with_subtotals_totals
  GROUP BY
    ALL
  ORDER BY
   date_month,
   account_report_category_order
  """

  account_group_query = f"""
  WITH
  report AS (
  SELECT
    date_month,
    CASE
      WHEN account_type = 'REVENUE' THEN 'Turnover'
      WHEN account_type = 'OVERHEADS' THEN 'Overheads'
      WHEN account_type = 'DIRECTCOSTS' THEN 'Direct Costs'
  END
    AS account_category,
    CASE
      WHEN account_type = 'REVENUE' THEN 1
      WHEN account_type = 'OVERHEADS' THEN 4
      WHEN account_type = 'DIRECTCOSTS' THEN 2
  END
    AS account_report_category_order,
    account_name AS account_group,
    account_id,
    account_code,
    account_class,
    account_type,
    COALESCE(SUM(net_amount* -1),0) AS net_amount
  FROM
    `{PROJECT_ID}.{DATASET_ID}.xero__profit_and_loss_report`
  GROUP BY
    1,
    2,
    3,
    4,
    5,
    6,
    7,
    8),
  spine AS (
  WITH
    rawdata AS (
    WITH
      p AS (
      SELECT
        0 AS generated_number
      UNION ALL
      SELECT
        1 ),
      unioned AS (
      SELECT
        p0.generated_number * POWER(2, 0) + p1.generated_number * POWER(2, 1) + p2.generated_number * POWER(2, 2) + p3.generated_number * POWER(2, 3) + p4.generated_number * POWER(2, 4) + p5.generated_number * POWER(2, 5) + p6.generated_number * POWER(2, 6) + 1 AS generated_number
      FROM
        p AS p0
      CROSS JOIN
        p AS p1
      CROSS JOIN
        p AS p2
      CROSS JOIN
        p AS p3
      CROSS JOIN
        p AS p4
      CROSS JOIN
        p AS p5
      CROSS JOIN
        p AS p6 )
    SELECT
      *
    FROM
      unioned
    WHERE
      generated_number <= 68
    ORDER BY
      generated_number ),
    all_periods AS (
    SELECT
      ( DATETIME_ADD( CAST( CAST('2023-06-01' AS date) AS datetime), INTERVAL ROW_NUMBER() OVER (ORDER BY 1) - 0 month ) ) AS date_month
    FROM
      rawdata ),
    filtered AS (
    SELECT
      *
    FROM
      all_periods
    WHERE
      date_month <= DATETIME_ADD( CAST( current_date AS datetime), INTERVAL 1 month ) )
  SELECT
    *
  FROM
    filtered ),
  cleaned AS (
  SELECT
    CAST(date_month AS date) AS date_month
  FROM
    spine ),
  calendar AS (
  SELECT
    *
  FROM
    cleaned),
  joined AS (
  SELECT
    calendar.date_month,
    report.account_id,
    report.account_group,
    report.account_code,
    report.account_type,
    report.account_class,
    report.account_category,
    COALESCE(SUM(net_amount * -1),0) AS net_amount
  FROM
    calendar
  LEFT JOIN
    report
  ON
    calendar.date_month = CAST( TIMESTAMP_TRUNC( CAST(report.date_month AS timestamp ), month ) AS date)
  GROUP BY
    1,2,3,4,5,6,7)
select * from joined
group by all
order by date_month, account_code
"""

  transactions_query = f"""
  WITH
    spine AS (
    WITH
      rawdata AS (
      WITH
        p AS (
        SELECT
          0 AS generated_number
        UNION ALL
        SELECT
          1 ),
        unioned AS (
        SELECT
          p0.generated_number * POWER(2, 0) + p1.generated_number * POWER(2, 1) + p2.generated_number * POWER(2, 2) + p3.generated_number * POWER(2, 3) + p4.generated_number * POWER(2, 4) + p5.generated_number * POWER(2, 5) + p6.generated_number * POWER(2, 6) + 1 AS generated_number
        FROM
          p AS p0
        CROSS JOIN
          p AS p1
        CROSS JOIN
          p AS p2
        CROSS JOIN
          p AS p3
        CROSS JOIN
          p AS p4
        CROSS JOIN
          p AS p5
        CROSS JOIN
          p AS p6 )
      SELECT
        *
      FROM
        unioned
      WHERE
        generated_number <= 68
      ORDER BY
        generated_number ),
      all_periods AS (
      SELECT
        ( DATETIME_ADD( CAST( CAST('2019-01-01' AS date) AS datetime), INTERVAL ROW_NUMBER() OVER (ORDER BY 1) - 0 month ) ) AS date_month
      FROM
        rawdata ),
      filtered AS (
      SELECT
        *
      FROM
        all_periods
      WHERE
        date_month <= DATETIME_ADD( CAST( current_date AS datetime), INTERVAL 1 month ) )
    SELECT
      *
    FROM
      filtered ),
    cleaned AS (
    SELECT
      CAST(date_month AS date) AS date_month
    FROM
      spine ),
    calendar AS (
    SELECT
      *
    FROM
      cleaned),
    ledger AS (
    SELECT
      *
    FROM
      `{PROJECT_ID}.{DATASET_ID}.xero__general_ledger` ),
    joined AS (
    SELECT
      TO_HEX(MD5(CAST(COALESCE(CAST(calendar.date_month AS string ), '') || '-' || COALESCE(CAST(ledger.account_id AS string ), '') AS string ))) AS profit_and_loss_pk,
      calendar.date_month,
      ledger.account_id,
      ledger.account_name as account_group,
      ledger.account_code,
      ledger.account_type,
      COALESCE(SUM(ledger.net_amount * -1),0) AS net_amount
    FROM
      calendar
    LEFT JOIN
      ledger
    ON
      calendar.date_month = CAST( TIMESTAMP_TRUNC( CAST(ledger.journal_date AS timestamp ), month ) AS date)
    WHERE
      ledger.account_class IN ('REVENUE',
        'EXPENSE')
    GROUP BY
      1,
      2,
      3,
      4,
      5,
      6),
    journals AS (
    SELECT
      DATE_TRUNC(journal_date,MONTH) AS date_month,
      account_code,
      journal_number,
      reference,
      source_type,
      account_name as account_group,
      account_type,
      description,
      gross_amount,
      net_amount,
      tax_amount
    FROM
      `{PROJECT_ID}.{DATASET_ID}.xero__general_ledger` ),
    account_groups AS (
    SELECT
      date_month,
      account_group,
      account_code,
      account_type,
      SUM(net_amount) AS net_amount
    FROM
      joined
    GROUP BY
      1,
      2,
      3,
      4)
  SELECT
    g.date_month,
    g.account_group,
    g.account_type,
    j.account_code,
    j.description,
    COALESCE(j.net_amount * -1,0) AS net_amount
  FROM
    account_groups g
  LEFT JOIN
    journals j
  ON
    g.account_code = j.account_code
    AND g.date_month = j.date_month
  ORDER BY
    1,
    2,
    3,
    4,
    5
"""
  data = fetch_data(client, pl_query)
  account_group_data = fetch_data(client, account_group_query)
  transactions_data = fetch_data(client, transactions_query)
  return data, account_group_data, transactions_data

In [9]:
# Analysis functions

def get_significant_transactions(transactions_data, account_group, date, threshold=0.1):
  current_month = transactions_data[(transactions_data['account_group'] == account_group) &
                   (transactions_data['date_month'] == date)]
  total_amount = current_month['net_amount'].sum()
  significant_transactions = current_month[abs(current_month['net_amount']) > abs(total_amount * threshold)]
  return significant_transactions.to_dict('records')

def calculate_percentage_change(current, previous):
  """Calculate percentage change, treating costs as positive values."""
  current_abs = abs(current)
  previous_abs = abs(previous)
  if previous_abs == 0:
    return float('inf') if current_abs > 0 else 0
  return ((current_abs - previous_abs) / previous_abs) * 100

In [10]:
def format_financial_change(current, previous, category_name):
  """Format financial change consistently, treating costs as positive values."""
  current_abs = abs(current)
  previous_abs = abs(previous)
  change = current_abs - previous_abs
  percentage_change = calculate_percentage_change(current, previous)

  if change > 0:
    change_direction = "increased"
  elif change < 0:
    change_direction = "decreased"
  else:
    change_direction = "remained unchanged"

  return f"{category_name} {change_direction} to £{current_abs:,.2f} in the current month, " \
      f"from £{previous_abs:,.2f} in the previous month, " \
      f"a change of {abs(percentage_change):.2f}%."



In [11]:
def analyze_cost_changes(transactions_data, account_group, date):
  current_month = transactions_data[(transactions_data['account_group'] == account_group) &
                   (transactions_data['date_month'] == date)]
  previous_month = transactions_data[(transactions_data['account_group'] == account_group) &
                    (transactions_data['date_month'] == date - pd.DateOffset(months=1))]

  current_total = current_month['net_amount'].sum()
  previous_total = previous_month['net_amount'].sum()
  total_change = abs(current_total) - abs(previous_total)

  category_changes = {}
  for category in ['Salary', 'Bonus', 'Dividend', 'Software/Subscription', 'Other']:
    current_category = current_month[current_month['description'].str.contains(category, case=False, na=False)]
    previous_category = previous_month[previous_month['description'].str.contains(category, case=False, na=False)]

    current_amount = current_category['net_amount'].sum()
    previous_amount = previous_category['net_amount'].sum()
    change = abs(current_amount) - abs(previous_amount)

    category_changes[category] = {
      'current_amount': current_amount,
      'previous_amount': previous_amount,
      'change': change,
      'percentage_change': calculate_percentage_change(current_amount, previous_amount),
      'formatted_change': format_financial_change(current_amount, previous_amount, category)
    }

  return {
    'total_change': total_change,
    'total_percentage_change': calculate_percentage_change(current_total, previous_total),
    'formatted_total_change': format_financial_change(current_total, previous_total, account_group),
    'category_changes': category_changes
  }

In [12]:
def analyze_overhead_trends(data, months=6):
  overhead_data = data[(data['account_category'] == 'Overheads') &
             (data['date_month'] >= data['date_month'].max() - pd.DateOffset(months=months))]

  grouped = overhead_data.groupby(['account_group', 'date_month'])['net_amount'].sum().unstack()
  growth_rates = grouped.pct_change(axis=1).mean(axis=1)

  significant_growth = growth_rates[growth_rates > 0.10]

  return significant_growth

def identify_cancelling_transactions(transactions):
  """
  Identify pairs of transactions that cancel each other out.
  """
  cancelling_pairs = []
  for i, trans1 in enumerate(transactions):
    for j, trans2 in enumerate(transactions[i+1:], start=i+1):
      if (trans1['account_group'] == trans2['account_group'] and
        trans1['description'] == trans2['description'] and
        trans1['net_amount'] == -trans2['net_amount']):
        cancelling_pairs.append((i, j))
  return cancelling_pairs

In [13]:
def get_account_group_details(account_group_data, transactions_data, category, date):
  current_month = account_group_data[(account_group_data['account_type'] == category) &
                    (account_group_data['date_month'] == date)]
  previous_month = account_group_data[(account_group_data['account_type'] == category) &
                    (account_group_data['date_month'] == date - pd.DateOffset(months=1))]

  group_totals = current_month.groupby('account_group')['net_amount'].sum().to_dict()
  prev_group_totals = previous_month.groupby('account_group')['net_amount'].sum().to_dict()

  group_details = {}
  for group, amount in group_totals.items():
    previous_amount = prev_group_totals.get(group, 0)
    significant_transactions = get_significant_transactions(transactions_data, group, date)

    group_details[group] = {
      'total': amount,
      'previous_total': previous_amount,
      'change': amount - previous_amount,
      'percentage_change': calculate_percentage_change(amount, previous_amount),
      'is_new': group not in prev_group_totals,
      'significant_transactions': significant_transactions
    }

  if category == 'EXPENSE':
    trend_analysis = analyze_overhead_trends(account_group_data)
    return {
      'group_details': group_details,
      'trend_analysis': trend_analysis.to_dict()
    }
  elif category in ['REVENUE', 'DIRECTCOSTS']:
    return group_details
  else:
    total = current_month['net_amount'].sum()
    previous_total = previous_month['net_amount'].sum()
    return {
      'total': total,
      'previous_total': previous_total,
      'change': total - previous_total,
      'percentage_change': calculate_percentage_change(total, previous_total)
    }

In [14]:
# Data Preparation

def process_data(data, account_group_data, transactions_data):
    data['date_month'] = pd.to_datetime(data['date_month'], format='%Y-%m-%d')
    data['net_amount'] = data['net_amount'].astype(float)

    unique_months = sorted(data['date_month'].unique())
    last_month = unique_months[-1]
    second_last_month = unique_months[-2]
    third_last_month = unique_months[-3]

    current_month_data = data[data['date_month'] == second_last_month]
    previous_month_data = data[data['date_month'] == third_last_month]
    two_months_ago_data = data[data['date_month'] == unique_months[-4]]

    current_year = second_last_month.year
    previous_year = current_year - 1

    ytd_data_current = data[(data['date_month'].dt.year == current_year) & (data['date_month'] <= second_last_month)]
    ytd_totals_current = ytd_data_current.groupby(['account_category', 'account_report_category_order'])['net_amount'].sum().reset_index()

    ytd_data_previous = data[(data['date_month'].dt.year == previous_year) & (data['date_month'] <= second_last_month.replace(year=previous_year))]
    ytd_totals_previous = ytd_data_previous.groupby(['account_category', 'account_report_category_order'])['net_amount'].sum().reset_index()

    report_data = current_month_data.merge(previous_month_data, on=['account_category', 'account_report_category_order'], suffixes=('', '_prev'), how='outer')
    report_data = report_data.merge(two_months_ago_data, on=['account_category', 'account_report_category_order'], suffixes=('', '_2mo_ago'), how='outer')
    report_data = report_data.merge(ytd_totals_current, on=['account_category', 'account_report_category_order'], suffixes=('', '_ytd'), how='outer')
    report_data = report_data.merge(ytd_totals_previous, on=['account_category', 'account_report_category_order'], suffixes=('', '_prev_ytd'), how='outer')

    report_data['pct_change_mom'] = (report_data['net_amount'] - report_data['net_amount_prev']) / abs(report_data['net_amount_prev'])
    report_data['pct_change_ytd'] = (report_data['net_amount_ytd'] - report_data['net_amount_prev_ytd']) / abs(report_data['net_amount_prev_ytd'])

    account_group_data['date_month'] = pd.to_datetime(account_group_data['date_month'], format='%Y-%m-%d')
    account_group_data['net_amount'] = account_group_data['net_amount'].astype(float)

    transactions_data['date_month'] = pd.to_datetime(transactions_data['date_month'], format='%Y-%m-%d')
    transactions_data['net_amount'] = transactions_data['net_amount'].astype(float)

    last_month_data = report_data[report_data['date_month'] == second_last_month]
    last_month_data = last_month_data[['account_category', 'net_amount', 'net_amount_prev', 'net_amount_ytd', 'net_amount_prev_ytd']]

    return report_data, second_last_month, third_last_month, unique_months, current_year, previous_year

In [15]:
def prepare_account_group_details(current_month_groups):
  account_group_details = ""
  for category in current_month_groups:
    details = current_month_groups[category]
    account_group_details += f"\n{category}:\n"

    if category == 'EXPENSE':
      for group, group_data in details['group_details'].items():
        current_amount = group_data['total']
        previous_amount = group_data['previous_total']
        change = group_data['change']
        percentage_change = group_data['percentage_change']
        is_new = group_data['is_new']

        if is_new:
          account_group_details += f" - NEW: {group}: {format_currency(current_amount)}\n"
        else:
          account_group_details += f" - {group}: {format_currency(current_amount)} (Previous: {format_currency(previous_amount)}, Change: {format_currency(change)}, {percentage_change:.2f}%)\n"

        if group_data['significant_transactions']:
          account_group_details += "  Significant Transactions:\n"
          for transaction in group_data['significant_transactions']:
            amount = transaction['net_amount']
            description = transaction['description']
            account_group_details += f"   * {format_currency(amount)} - {description}\n"

      account_group_details += "\nOverhead Trend Analysis (Past 6 Months):\n"
      for group, growth_rate in details['trend_analysis'].items():
        account_group_details += f" - {group}: {growth_rate:.2%} average monthly growth\n"

    elif category in ['REVENUE', 'DIRECTCOSTS']:
      for group, group_data in details.items():
        current_amount = group_data['total']
        previous_amount = group_data['previous_total']
        change = group_data['change']
        percentage_change = group_data['percentage_change']
        is_new = group_data['is_new']

        if is_new:
          account_group_details += f" - NEW: {group}: {format_currency(current_amount)}\n"
        else:
          account_group_details += f" - {group}: {format_currency(current_amount)} (Previous: {format_currency(previous_amount)}, Change: {format_currency(change)}, {percentage_change:.2f}%)\n"

        if group_data.get('significant_transactions'):
          account_group_details += "  Significant Transactions:\n"
          for transaction in group_data['significant_transactions']:
            amount = transaction['net_amount']
            description = transaction['description']
            account_group_details += f"   * {format_currency(amount)} - {description}\n"

    else:
      current_amount = details['total']
      previous_amount = details['previous_total']
      change = details['change']
      percentage_change = details['percentage_change']
      account_group_details += f" Current Month Total: {format_currency(current_amount)}\n"
      account_group_details += f" Previous Month Total: {format_currency(previous_amount)}\n"
      account_group_details += f" Change: {format_currency(change)} ({percentage_change:.2f}%)\n"

    account_group_details += "\n"

  return account_group_details

In [16]:
def format_cost_change_analysis(cost_change_analysis):
    """Convierte el dict de análisis de costos en un texto legible."""
    lines = [cost_change_analysis['formatted_total_change'], "\n"]

    for category, data in cost_change_analysis['category_changes'].items():
        lines.append(data['formatted_change'])

    return "\n".join(lines)


In [17]:
def generate_analysis_prompt(month, financial_data, account_group_details, cost_change_analysis):
    return PromptTemplate.from_template(f"""
You are a financial analyst writing a management report based on the following Profit and Loss data for {month}.

Your goal is to deliver an analysis that is:
- Easy to understand by non-financial readers
- Focused on clarity and actionable insights
- Professional, yet written in a human and approachable tone

DATA INPUT:

 FINANCIAL DATA:
{financial_data}

 ACCOUNT GROUP DETAILS:
{account_group_details}

 COST CHANGE ANALYSIS:
{cost_change_analysis}

❗ Avoid technical jargon or overly complex language. Use clear and simple terms, short paragraphs, and highlight the most important patterns, shifts, and risks. Provide explanations as if speaking to a smart manager with limited accounting background.

Focus on:
1. What changed from last month — and why
2. What new costs or revenues appeared — and their possible meaning
3. Which areas should be monitored or optimized
4. How this month compares to the same period last year (YTD vs LYTD)
5. Whether the company’s financial health is improving or worsening

DO NOT speculate on things not shown in the data. DO NOT mention individual people (e.g., salaries of specific staff). Treat cost values as positive amounts even if they’re negative in accounting.

When describing changes, follow this structure:
- “Revenue increased to £X in April 2025, compared to £Y in March 2025, a change of Z%.”
- “A new expense for [category] appeared, amounting to £X.”
- “The expense for [category] increased/decreased to £X, compared to an average of £Y over the past 3 months.”

At the end, include a short summary of recommendations for management.

 OUTPUT FORMAT (Use this exact HTML structure):

<h1> Executive Summary</h1>
<ul>
 <li><b> Summary:</b> [One clear sentence about overall performance and direction]</li>
 <li><b> Recommendation:</b> [Top insight or action management should focus on]</li>
</ul>

<h2> Revenue</h2>
<p>[Clear explanation of revenue change and driver]</p>

<h3> Direct Costs</h3>
<p>[Details on cost changes, new items, or unusual shifts]</p>

<h3> Gross Margin</h3>
<p>[Explanation of impact of revenues and costs on gross margin]</p>

<h3> Overheads</h3>
<p>[Highlight key overheads that changed significantly, and suggest potential review if needed]</p>

<h2> Net Profit and Retained Earnings</h2>
<p>[Final paragraph tying everything together: is the company more or less profitable than last month? What changed and what could be improved?]</p>

<h2> Management Insights</h2>
<ul>
 <li>[#1 Actionable insight or recommendation]</li>
 <li>[#2 Actionable insight or recommendation]</li>
</ul>
""")





In [18]:
def generate_financial_data_string(report_data, second_last_month, third_last_month, current_year, previous_year):
  """Generate a string containing financial data for the LLM prompt."""

  def get_financial_value(df, category, column):
    filtered_df = df[df['account_category'] == category]
    if not filtered_df.empty:
      return filtered_df[column].values[0]
    return 0

  financial_data = f"""
  Last Month's Performance ({second_last_month.strftime('%B %Y')}):
  Revenue: {format_currency(get_financial_value(report_data, 'Turnover', 'net_amount'))}
  Gross Margin %: {safe_percentage(get_financial_value(report_data, 'Gross Profit', 'net_amount'), get_financial_value(report_data, 'Turnover', 'net_amount')):.2f}%
  Retained Earnings: {format_currency(get_financial_value(report_data, 'Retained Earnings', 'net_amount'))}
  Direct Costs: {format_currency(get_financial_value(report_data, 'Direct Costs', 'net_amount'))}
  Overheads: {format_currency(get_financial_value(report_data, 'Overheads', 'net_amount'))}
  Net Profit: {format_currency(get_financial_value(report_data, 'Net Profit', 'net_amount'))}

  Previous Month's Performance ({third_last_month.strftime('%B %Y')}):
  Revenue: {format_currency(get_financial_value(report_data, 'Turnover', 'net_amount_prev'))}
  Gross Margin %: {safe_percentage(get_financial_value(report_data, 'Gross Profit', 'net_amount_prev'), get_financial_value(report_data, 'Turnover', 'net_amount_prev')):.2f}%
  Retained Earnings: {format_currency(get_financial_value(report_data, 'Retained Earnings', 'net_amount_prev'))}
  Direct Costs: {format_currency(get_financial_value(report_data, 'Direct Costs', 'net_amount_prev'))}
  Overheads: {format_currency(get_financial_value(report_data, 'Overheads', 'net_amount_prev'))}
  Net Profit: {format_currency(get_financial_value(report_data, 'Net Profit', 'net_amount_prev'))}

  Year-to-Date Performance:
  Revenue: {format_currency(get_financial_value(report_data, 'Turnover', 'net_amount_ytd'))}
  Gross Margin %: {safe_percentage(get_financial_value(report_data, 'Gross Profit', 'net_amount_ytd'), get_financial_value(report_data, 'Turnover', 'net_amount_ytd')):.2f}%
  Retained Earnings: {format_currency(get_financial_value(report_data, 'Retained Earnings', 'net_amount_ytd'))}
  Direct Costs: {format_currency(get_financial_value(report_data, 'Direct Costs', 'net_amount_ytd'))}
  Overheads: {format_currency(get_financial_value(report_data, 'Overheads', 'net_amount_ytd'))}
  Net Profit: {format_currency(get_financial_value(report_data, 'Net Profit', 'net_amount_ytd'))}

  Previous Year's Year-to-Date Performance:
  Revenue: {format_currency(get_financial_value(report_data, 'Turnover', 'net_amount_prev_ytd'))}
  Gross Margin %: {safe_percentage(get_financial_value(report_data, 'Gross Profit', 'net_amount_prev_ytd'), get_financial_value(report_data, 'Turnover', 'net_amount_prev_ytd')):.2f}%
  Retained Earnings: {format_currency(get_financial_value(report_data, 'Retained Earnings', 'net_amount_prev_ytd'))}
  Direct Costs: {format_currency(get_financial_value(report_data, 'Direct Costs', 'net_amount_prev_ytd'))}
  Overheads: {format_currency(get_financial_value(report_data, 'Overheads', 'net_amount_prev_ytd'))}
  Net Profit: {format_currency(get_financial_value(report_data, 'Net Profit', 'net_amount_prev_ytd'))}
  """
  return financial_data

In [19]:
def generate_report_html(report_data, second_last_month, third_last_month, unique_months, current_year, previous_year):
  revenue_current = report_data[report_data['account_category'] == 'Turnover']['net_amount'].values[0]
  revenue_prev = report_data[report_data['account_category'] == 'Turnover']['net_amount_prev'].values[0]
  revenue_2mo_ago = report_data[report_data['account_category'] == 'Turnover']['net_amount_2mo_ago'].values[0]
  revenue_ytd = report_data[report_data['account_category'] == 'Turnover']['net_amount_ytd'].values[0]
  revenue_prev_ytd = report_data[report_data['account_category'] == 'Turnover']['net_amount_prev_ytd'].values[0]

  report_data_html = f"""
  <h2>Profit & Loss Report Detail with Period-on-Period Comparisons</h2>
  <table class="financial-table">
    <tr>
      <th>Category</th>
      <th>{second_last_month.strftime('%b %Y')}</th>
      <th>% Change MoM</th>
      <th>{third_last_month.strftime('%b %Y')}</th>
      <th>{unique_months[-4].strftime('%b %Y')}</th>
      <th>YTD {current_year}</th>
      <th>% Change YTD</th>
      <th>YTD {previous_year}</th>
    </tr>
  """

  for _, row in report_data.iterrows():
    category = row['account_category']
    current_amount = row['net_amount']
    pct_change_mom = row['pct_change_mom']
    previous_amount = row['net_amount_prev']
    two_months_ago_amount = row['net_amount_2mo_ago']
    ytd_amount = row['net_amount_ytd']
    pct_change_ytd = row['pct_change_ytd']
    prev_ytd_amount = row['net_amount_prev_ytd']

    class_name = 'category'
    if category in ['Direct Costs', 'Overheads', 'Taxation']:
      class_name = 'subcategory'

    report_data_html += f"""
    <tr class="{class_name}">
      <td>{category}</td>
      <td>{format_currency(current_amount)}</td>
      <td>{pct_change_mom:.2%}</td>
      <td>{format_currency(previous_amount)}</td>
      <td>{format_currency(two_months_ago_amount)}</td>
      <td>{format_currency(ytd_amount)}</td>
      <td>{pct_change_ytd:.2%}</td>
      <td>{format_currency(prev_ytd_amount)}</td>
    </tr>
    """

    if category in ['Direct Costs', 'Gross Profit', 'Overheads', 'Net Profit', 'Retained Earnings']:
      current_percentage = safe_percentage(current_amount, revenue_current)
      prev_percentage = safe_percentage(previous_amount, revenue_prev)
      two_months_ago_percentage = safe_percentage(two_months_ago_amount, revenue_2mo_ago)
      ytd_percentage = safe_percentage(ytd_amount, revenue_ytd)
      prev_ytd_percentage = safe_percentage(prev_ytd_amount, revenue_prev_ytd)

      report_data_html += f"""
      <tr class="{class_name}-percentage">
        <td>{category} as % of Sales</td>
        <td>{current_percentage:.2f}%</td>
        <td></td>
        <td>{prev_percentage:.2f}%</td>
        <td>{two_months_ago_percentage:.2f}%</td>
        <td>{ytd_percentage:.2f}%</td>
        <td></td>
        <td>{prev_ytd_percentage:.2f}%</td>
      </tr>
      """

  report_data_html += "</table>"

  return report_data_html

In [20]:
def generate_full_html_output(report_analysis_html, report_data_html):
  """Generate the full HTML output for display in the notebook."""

  return f"""
  <html>
  <head>
  <style>
    body {{
      font-family: Arial, sans-serif;
      line-height: 1.6;
      color: #333;
    }}
    h1, h2, h3 {{
      color: #2c3e50;
    }}
    table {{
      border-collapse: collapse;
      width: 100%;
	margin-top: 20px;
    }}
    th, td {{
      border: 1px solid #ddd;
      padding: 8px;
      text-align: right;
    }}
    th {{
      background-color: #f2f2f2;
      color: #2c3e50;
    }}
    .category {{
      text-align: left;
      font-weight: bold;
    }}
    .subcategory {{
      padding-left: 20px;
    }}
    .percentage {{
      font-style: italic;
      color: #555;
    }}
    .analysis {{
      border: 1px solid #ddd;
      padding: 15px;
      margin-bottom: 20px;
    }}
    .analysis h2 {{
      color: #2c3e50;
      padding-bottom: 10px;
    }}
    .analysis h3 {{
      color: #2c3e50;
    }}
    .analysis ul {{
      padding-left: 0;
    }}
    .analysis li {{
      margin-bottom: 10px;
    }}
    .executive-summary li {{
      font-weight: bold;
    }}
   </style>
  </head>
  <body>
  <h1>📊 Profit and Loss Report</h1>
  {report_data_html}
  <div class="analysis">
  {report_analysis_html}
  </div>
  </body>
  </html>
  """



In [21]:
def write_report_to_bigquery(client, date_month, report_data, report_analysis):
  """Write the generated report to BigQuery."""
  dataset_id = 'fivetran_demo_xero_xero'
  table_id = 'pl_reports_demo'

  dataset_ref = client.dataset(dataset_id)
  table_ref = dataset_ref.table(table_id)

  schema = [
    bigquery.SchemaField("date_month", "DATE", mode="REQUIRED"),
    bigquery.SchemaField("report_created_ts", "TIMESTAMP", mode="REQUIRED"),
    bigquery.SchemaField("report_data", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("report_analysis", "STRING", mode="REQUIRED"),
  ]

  try:
    client.get_dataset(dataset_ref)
  except exceptions.NotFound:
    print(f"Dataset {dataset_id} not found. Creating dataset.")
    dataset = bigquery.Dataset(dataset_ref)
    dataset.location = LOCATION
    client.create_dataset(dataset)
    print(f"Dataset {dataset_id} created.")

  table = bigquery.Table(table_ref, schema=schema)
  table = client.create_table(table, exists_ok=True)

  rows_to_insert = [
    (date_month, datetime.now(), report_data, report_analysis)
  ]

  errors = client.insert_rows(table, rows_to_insert)
  if errors == []:
    print(f"Report for {date_month} successfully written to BigQuery")
  else:
    print(f"Errors occurred while inserting rows: {errors}")

In [22]:
def main():
  bigquery_client, llm = initialize_clients()

  data, account_group_data, transactions_data = get_pl_data(bigquery_client)

  report_data, second_last_month, third_last_month, unique_months, current_year, previous_year = process_data(data, account_group_data, transactions_data)

  current_month_groups = {category: get_account_group_details(account_group_data, transactions_data, category, second_last_month)
              for category in ['REVENUE', 'DIRECTCOSTS', 'EXPENSE']}
  account_group_details = prepare_account_group_details(current_month_groups)

  cost_change_analysis = analyze_cost_changes(transactions_data, 'Direct Costs', second_last_month)

  financial_data = generate_financial_data_string(report_data, second_last_month, third_last_month, current_year, previous_year)

  cost_change_text = format_cost_change_analysis(cost_change_analysis)

  analysis_prompt = generate_analysis_prompt(
    second_last_month.strftime('%B %Y'),
    financial_data,
    account_group_details,
    cost_change_text
  )

  try:
    report_analysis_html = llm(analysis_prompt.format(
      month=second_last_month.strftime('%B %Y'),
      financial_data=financial_data,
      account_group_details=account_group_details,
      cost_change_analysis=cost_change_analysis
    ))
  except Exception as e:
    report_analysis_html = f"Error generating analysis: {html.escape(str(e))}"

  report_data_html = generate_report_html(report_data, second_last_month, third_last_month, unique_months, current_year, previous_year)
  report_data_html = re.sub(r'\*\*(.+?)\*\*', r'<b>\1</b>', report_data_html)

  full_html_output = generate_full_html_output(
    report_analysis_html.replace("**", ""),
    report_data_html.replace("**", "")
  )

  display(HTML(full_html_output))

  write_report_to_bigquery(bigquery_client, second_last_month.date(), report_data_html.replace("**", ""), report_analysis_html.replace("**", ""))

if __name__ == "__main__":
  main()


<ipython-input-22-dbeff5975bc6>:26: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  report_analysis_html = llm(analysis_prompt.format(


Category,Apr 2025,% Change MoM,Mar 2025,Feb 2025,YTD 2025,% Change YTD,YTD 2024
Direct Costs,"£-1,500.00",nan%,£nan,£nan,"£-1,500.00",nan%,£nan
Direct Costs as % of Sales,-18.49%,,nan%,nan%,-6.41%,,nan%
Gross Profit,"£6,614.15",-26.13%,"£8,954.03","£5,871.85","£21,897.70",4684.60%,£457.67
Gross Profit as % of Sales,81.51%,,100.00%,100.00%,93.59%,,100.00%
Net Profit,"£6,614.15",-26.13%,"£8,954.03","£5,871.85","£21,897.70",4684.60%,£457.67
Net Profit as % of Sales,81.51%,,100.00%,100.00%,93.59%,,100.00%
Retained Earnings,"£4,960.61",-26.13%,"£6,715.52","£4,403.89","£16,423.28",4684.60%,£343.25
Retained Earnings as % of Sales,61.14%,,75.00%,75.00%,70.19%,,75.00%
Taxation,"£1,653.54",-26.13%,"£2,238.51","£1,467.96","£5,474.43",4684.60%,£114.42
Turnover,"£8,114.15",-9.38%,"£8,954.03","£5,871.85","£23,397.70",5012.35%,£457.67


Report for 2025-04-01 successfully written to BigQuery
